<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
import sys
sys.path.append('../src')
sys.path.append('../utils')

import torch
import time
import numpy as np
from sigKer_torch import SigLoss, SigKernel, SigLoss_naive, SigKernel_naive

In [2]:
n=0

# Sig kernel gradients

In [3]:
A,M,N,D,var = 5, 20, 50, 2, 1e-1
X = var*torch.randn((A,M,D), dtype=torch.double)
Y = var*torch.randn((A,N,D), dtype=torch.double)
X.requires_grad = True

In [4]:
t = time.time()
ker_naive = SigKernel_naive(X,Y,n)
print('time:', np.round(time.time()-t,3), 's')

time: 0.198 s


In [5]:
t = time.time()
ker = SigKernel.apply(X,Y,n)
print('time:', np.round(time.time()-t,3), 's')

time: 0.005 s


In [6]:
torch.abs(ker.sum() - ker_naive.sum()) < 1e-6

tensor(True)

In [7]:
l = ker_naive.mean()
t = time.time()
l.backward()
print('time:', np.round(time.time()-t,3), 's')
ker_grad_naive = X.grad

time: 0.297 s


In [8]:
X.grad.data.zero_()
t = time.time()
l = ker.mean()
l.backward()
# b.backward()
print('time:', np.round(time.time()-t,3), 's')
ker_grad = X.grad

time: 0.001 s


In [9]:
torch.abs(ker_grad.sum() - ker_grad_naive.sum()) < 1e-6

tensor(True)

# Sig Loss gradients

In [10]:
A,M,N,D,var = 5, 20, 50, 2, 1e-1
X = var*torch.randn((A,M,D), dtype=torch.double)
Y = var*torch.randn((A,N,D), dtype=torch.double)
X.requires_grad = True

In [11]:
t = time.time()
l_naive = SigLoss_naive(n).forward(X,Y)
print('time:', np.round(time.time()-t,3), 's')

t = time.time()
l = SigLoss(n).forward(X,Y)
print('time:', np.round(time.time()-t,3), 's')

torch.abs(l.sum() - l_naive.sum()) < 1e-6

time: 0.586 s
time: 0.007 s


tensor(True)

In [12]:
t = time.time()
l_naive.backward()
print('time:', np.round(time.time()-t,3), 's')
l_grad_naive = X.grad

time: 0.433 s


In [13]:
X.grad.data.zero_()
t = time.time()
l.backward()
print('time:', np.round(time.time()-t,3), 's')
l_grad = X.grad

time: 0.0 s


In [14]:
torch.abs(l_grad.sum() - l_grad_naive.sum()) < 1e-6

tensor(True)